---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [10]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [11]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    colleges = open('university_towns.txt')
    state =""
    region=""
    #rg = list()
    data= pd.DataFrame(columns=["State", "RegionName"])
    #print(data)
    for lines in colleges:
        lines = lines.rstrip()
        if "[edit" in lines:
            state= lines.replace("[edit]","")
        else:
            #rg.append(lines)
            region = lines
            data = data.append({"State":state,"RegionName":region},ignore_index=True)
    data['RegionName']=data['RegionName'].str.replace('\s\(.*','')
    return data

In [12]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    quater_wise = pd.read_excel('gdplev.xls',skiprows=5,index_col=6)
    #quater_wise=quater_wise.reset_index()
    #quater_wise=quater_wise.drop(['GDP in billions of current dollars','GDP in billions of chained 2009 dollars'])
    quater_wise.columns.values.tolist()
    quater_wise=quater_wise.drop(['Unnamed: 0','GDP in billions of current dollars','GDP in billions of chained 2009 dollars','Unnamed: 3','Unnamed: 7'],axis=1)
    #quater_wise=quater_wise.reset_index
    #quater_wise=quater_wise.drop([0,1])
    quater_wise.drop(quater_wise.index[:2], inplace=True)
    quater_wise.rename(columns={'Unnamed: 4':'quater'},inplace=True)
    quater_wise.reset_index(inplace=True)
    quater_wise.set_index('quater',inplace=True)
    chained = quater_wise['GDP in billions of chained 2009 dollars.1']
    start=chained.index.values.tolist().index("2000q1")
    end=start+5
    for end in range(218,len(chained)):
        if (chained[start]>chained[start+1]) and (chained[start+1]>chained[start+2]) and (chained[start+2]>chained[start+3]) and (chained[start+3]<chained[start+4]) and (chained[start+4]<chained[start+5]):
            rec=chained[start:end]       
        start += 1
    return rec.index.values.tolist()[0]

In [13]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    quater_wise = pd.read_excel('gdplev.xls',skiprows=5,index_col=6)
    #quater_wise=quater_wise.reset_index()
    #quater_wise=quater_wise.drop(['GDP in billions of current dollars','GDP in billions of chained 2009 dollars'])
    quater_wise.columns.values.tolist()
    quater_wise=quater_wise.drop(['Unnamed: 0','GDP in billions of current dollars','GDP in billions of chained 2009 dollars','Unnamed: 3','Unnamed: 7'],axis=1)
    #quater_wise=quater_wise.reset_index
    #quater_wise=quater_wise.drop([0,1])
    quater_wise.drop(quater_wise.index[:2], inplace=True)
    quater_wise.rename(columns={'Unnamed: 4':'quater'},inplace=True)
    quater_wise.reset_index(inplace=True)
    quater_wise.set_index('quater',inplace=True)
    chained = quater_wise['GDP in billions of chained 2009 dollars.1']
    start=chained.index.values.tolist().index("2000q1")
    end=start+5
    for end in range(218,len(chained)):
        if (chained[start]>chained[start+1]) and (chained[start+1]>chained[start+2]) and (chained[start+2]>chained[start+3]) and (chained[start+3]<chained[start+4]) and (chained[start+4]<chained[start+5]):
            rec=chained[start:end]       
        start += 1
    return rec.index.values.tolist()[-1]

In [14]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    quater_wise = pd.read_excel('gdplev.xls',skiprows=5,index_col=6)
    #quater_wise=quater_wise.reset_index()
    #quater_wise=quater_wise.drop(['GDP in billions of current dollars','GDP in billions of chained 2009 dollars'])
    quater_wise.columns.values.tolist()
    quater_wise=quater_wise.drop(['Unnamed: 0','GDP in billions of current dollars','GDP in billions of chained 2009 dollars','Unnamed: 3','Unnamed: 7'],axis=1)
    #quater_wise=quater_wise.reset_index
    #quater_wise=quater_wise.drop([0,1])
    quater_wise.drop(quater_wise.index[:2], inplace=True)
    quater_wise.rename(columns={'Unnamed: 4':'quater'},inplace=True)
    quater_wise.reset_index(inplace=True)
    quater_wise.set_index('quater',inplace=True)
    chained = quater_wise['GDP in billions of chained 2009 dollars.1']
    start=chained.index.values.tolist().index("2000q1")
    end=start+5
    for end in range(218,len(chained)):
        if (chained[start]>chained[start+1]) and (chained[start+1]>chained[start+2]) and (chained[start+2]>chained[start+3]) and (chained[start+3]<chained[start+4]) and (chained[start+4]<chained[start+5]):
            rec=chained[start:end]       
        start += 1
    return rec.index.values.tolist()[-3]

In [15]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    housing_data=pd.read_csv('City_Zhvi_AllHomes.csv')
    #states #=pd.DataFrame(states,index=[0])
    ls = list()
    acro = list()
    for s in housing_data['State']:
        ls.append(states.get(s))
    housing_data['State'] = ls
    #list(housing_data.columns.values)
    housing_data.drop([ '1996-04',
     '1996-05',
     '1996-06',
     '1996-07',
     '1996-08',
     '1996-09',
     '1996-10',
     '1996-11',
     '1996-12',
     '1997-01',
     '1997-02',
     '1997-03',
     '1997-04',
     '1997-05',
     '1997-06',
     '1997-07',
     '1997-08',
     '1997-09',
     '1997-10',
     '1997-11',
     '1997-12',
     '1998-01',
     '1998-02',
     '1998-03',
     '1998-04',
     '1998-05',
     '1998-06',
     '1998-07',
     '1998-08',
     '1998-09',
     '1998-10',
     '1998-11',
     '1998-12',
     '1999-01',
     '1999-02',
     '1999-03',
     '1999-04',
     '1999-05',
     '1999-06',
     '1999-07',
     '1999-08',
     '1999-09',
     '1999-10',
     '1999-11',
     '1999-12'],axis=1,inplace=True)
    #list(housing_data.columns.values)
    df = housing_data[['RegionID', 'RegionName','State','Metro','CountyName','SizeRank']]
    df.set_index('RegionID',inplace=True)
    housing_data.set_index(['State','RegionName'],inplace=True)
    housing_data=housing_data.drop(['RegionID','Metro','CountyName','SizeRank'],axis=1)
    housing_data=housing_data.groupby(pd.PeriodIndex(housing_data.columns,freq='Q'),axis=1).mean()
    housing_data.columns = housing_data.columns.to_series().astype(str)
    housing_data = housing_data.rename(columns=lambda x: str(x).lower())
    return housing_data

In [16]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    df=convert_housing_data_to_quarters()
    #price_ratio= pd.DataFrame(df['2008Q2'].div(df['2009Q2']))
    price_ratio =  pd.DataFrame({'ratio': df['2008q2'].div(df['2009q2'])})
    # df = pd.concat([df, price_ratio], axis=1)
    uni=get_list_of_university_towns()
    uni_list=uni.to_records(index=False).tolist()
    uni_town = price_ratio.loc[price_ratio.index.isin(uni_list)]
    #uni_town = pd.to_numeric(uni_town['ratio'])
    Non_uni_town=price_ratio.loc[~price_ratio.index.isin(uni_list)]
    #Non_uni_town = pd.to_numeric(Non_uni_town['ratio'])
    #uni_town['ratio']
    a=ttest_ind(uni_town['ratio'], Non_uni_town['ratio'],nan_policy='omit')
    pval=a.pvalue
    b=uni_town['ratio'].mean()
    c=Non_uni_town['ratio'].mean()
    if b<c:
        better = 'university town'
    else:
        better = 'non-university town'
    difference = pval<0.01
    d=(difference,pval,better)
    #price_ratio
    #Non_uni_town.shape    
    return d

(True, 0.0027240637047531249, 'university town')